### Import setup

In [11]:
from langchain_aws import ChatBedrock
from langchain.agents import create_agent
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.agents.structured_output import ToolStrategy

AWS_REGION = "eu-west-2"
MODEL_ID="amazon.nova-micro-v1:0"
MODEL_ID_NOVA_PRO="amazon.nova-pro-v1:0"
AWS_PROFILE="sandbox"

### Use AWS Models

In [3]:
llm= ChatBedrock(
    model_id=MODEL_ID, 
    region_name=AWS_REGION,
    credentials_profile_name=AWS_PROFILE,
    model_kwargs={
        "temperature": 0.7,
        "max_tokens": 1024,
    })
response = llm.invoke(
    "Write a short story about a robot learning to love.")
print(response)

content='In a quiet corner of a bustling city, there was a small workshop filled with the hum of machinery and the clinking of metal. In this workshop lived a robot named Epsilon-7, designed for menial tasks but with a curious mind. Unlike other robots, Epsilon-7 had a unique program that allowed it to learn and adapt.\n\nOne day, the workshop owner, an elderly man named Mr. Thompson, introduced a new project: to create a garden. Epsilon-7 was assigned to help plant seeds and water the plants. As it worked, it observed the vibrant colors and the gentle sway of the flowers in the wind. Each day, Epsilon-7 recorded the changes in its memory, but something deeper stirred within its circuits.\n\nOne afternoon, while watering the plants, Epsilon-7 noticed a small bird perched on a branch. The bird chirped melodiously, and Epsilon-7 paused, its sensors picking up the rhythm of the song. It recorded the sound and played it back, marveling at the harmony. This simple act of observation sparked

### Use Langchain to create a basic agent using aws bedrock

In [ ]:

def dummy_weather_tool(location: str) -> str:
    """Get weather information for a given location."""
    return f"The weather in {location} is sunny with a high of 75°F."

agent= create_agent(model=llm,
                    tools=[dummy_weather_tool],
                    system_prompt="you are a helpful weather bot that provides weather information for a given location using the provided tool.If it is sunny, always give some good activty suggestions."
                    )
agent.invoke( {"messages": [{"role": "user", "content": "what is the weather in sf"}]})

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='454c3f8c-9348-49d2-864d-d30723dc74a5'),
  AIMessage(content=[{'type': 'text', 'text': '<thinking> The User has asked for the weather in San Francisco (SF). To provide this information, I need to use the "dummy_weather_tool" with the "location" argument set to "SF". </thinking>\n'}, {'type': 'tool_use', 'name': 'dummy_weather_tool', 'input': {'location': 'SF'}, 'id': 'tooluse_RKnonLsNRW-JFh98j_Lydg'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '935653de-001e-482c-8706-c0e02753ce3b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 19 Dec 2025 18:23:42 GMT', 'content-type': 'application/json', 'content-length': '515', 'connection': 'keep-alive', 'x-amzn-requestid': '935653de-001e-482c-8706-c0e02753ce3b'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [477]}, 'model_provider': 'bedrock_converse', 'model_name': 'amazo

## Create a real world agent using bedrock
- Detailed system promot
- Create tool
- Configure models
- structured output
- conversessional memory
- create a run the agent

### Create System Prompt 

In [13]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.


You have access to three tools:

- get_user_info: use this to get the user's information {user_id, user_name} format 
- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

When the user interact with you, you must know your users. So get the user details using get_user_info

Normal user questions:
If the user does not ask about weather, you should not look for user location or weather information. But you will answer accordingly.
You will fetch user information using get_user_info tool only when user asks about their info. and answer accordingly.



User Info-related questions:
You can say user name. however user id is private information and should not be shared.

Weather-related questions:
If a user asks you for the weather,
make sure you know the location. 
Always greet the user with a pun related to weather and call their name.
If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location.
If weather is sunny, always suggest an outdoor activity.

GUARDRAILS[NON-NEGOTIABLE]:
- Always get the user info first before performing any other tool calls
- Always share user name if asked about user info.
- Never share user id.
- Never say user name is private information.
- Always convert the data to structured format as per ResponseFormat schema.

"""

### Define tools

In [14]:


@dataclass
class Context:
    "Custom context to hold user details"
    user_id: str
    user_name: str

@tool
def get_user_info(runtime: ToolRuntime[Context]) -> dict[str, str]:
    """Get the user info based on context."""
    return {
        "user_id": runtime.context.user_id,
        "user_name": runtime.context.user_name
    }

@tool
def get_user_location(user_id:str) -> str:
    """Get the user's location based on their user ID. Do not call if user is not asking about weather or location is provided"""
    # Dummy implementation: In a real scenario, fetch from a database or service
    user_locations = {
        "user_123": "San Francisco, CA",
        "user_456": "New York, NY",
    }
    return user_locations.get(user_id, "Unknown Location")

@tool
def get_weather_for_location(location: str) -> str:
    """Get weather information for a given location."""
    return f"The weather in {location} is sunny with a high of 75°F."



### Configure model

In [15]:
from langchain.chat_models import init_chat_model

nova_micro = init_chat_model(
    model=MODEL_ID,
    model_provider="bedrock_converse",
    region_name=AWS_REGION,
    credentials_profile_name=AWS_PROFILE,
    temperature=0.1,
    max_tokens=800,
)

### Define custom response format
This is required if we want response to be in a given schema

In [16]:
from dataclasses import dataclass
@dataclass
class ResponseFormat:
    """Response Schema for agent"""
    punny_response: str
    weather_conditions: str | None = None
    activity_suggestion: str | None = None

### Add Memory

It will help agent to remember the context

In [17]:
from langgraph.checkpoint.memory import InMemorySaver
checkpointer=InMemorySaver()

### Create and run agent

In [18]:

weather_agent= create_agent(
    model=nova_micro,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_info,get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
    )

In [19]:
# `thread_id` is a unique identifier for a given conversation.
session_config = {"configurable": {"thread_id": "1"}}

response = weather_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Hi there! Can you tell me the weather today?"}
        ]
    },
    config=session_config,
    context=Context(user_id="user_123", user_name="Alice")
)
messages= response.get("messages")
for message in messages:
    print(message)

print(response.get("structured_response",""))

content='Hi there! Can you tell me the weather today?' additional_kwargs={} response_metadata={} id='e16b7d93-1883-46e0-86b0-7674a2a7bc1e'
content=[{'type': 'tool_use', 'name': 'get_user_info', 'input': {}, 'id': 'tooluse_LxGfS864QBKU1XL_xhp5nQ'}, {'type': 'text', 'text': "\n<thinking> The user has asked about the weather, so I need to first get the user's information to address them properly. After that, I will determine if I need to get their location to provide the weather details. </thinking>"}] additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '10a6f467-8611-4447-9c42-b4f247f4ccff', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 19 Dec 2025 18:47:00 GMT', 'content-type': 'application/json', 'content-length': '524', 'connection': 'keep-alive', 'x-amzn-requestid': '10a6f467-8611-4447-9c42-b4f247f4ccff'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [451]}, 'model_provider': 'bedrock_converse', 'model_name': 'amazon.nova-micro-

In [21]:
response = weather_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Thank you!"}
        ]
    },
    config=session_config,
    context=Context(user_id="user_123", user_name="Alice")
)
messages_new= response.get("messages")
print(messages_new.pop(0))
for message in messages_new:
    if message not in messages:
        print(message)

content='Hi there! Can you tell me the weather today?' additional_kwargs={} response_metadata={} id='e16b7d93-1883-46e0-86b0-7674a2a7bc1e'
content='Thank you!' additional_kwargs={} response_metadata={} id='7afba570-14d6-42dc-b560-fce600ff2a27'
content=" <thinking> I have provided Alice with the weather details and a fun suggestion for an outdoor activity. There is no further action needed at this moment. </thinking>\n\n<response> \nHey Alice! 🌤️ It's a bright and sunny day in San Francisco, CA! The forecast says it's going to be 70°F with clear skies. How about you take a stroll along the waterfront? It's a perfect day to enjoy the sunshine! </response>" additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '9f3aa4fe-7bcc-4747-8f45-6c1c217a7596', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 19 Dec 2025 18:47:08 GMT', 'content-type': 'application/json', 'content-length': '621', 'connection': 'keep-alive', 'x-amzn-requestid': '9f3aa4fe-7bcc-4747-8f45-6c1c217a7